In [10]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta
from dateutil.relativedelta import relativedelta

In [11]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\output data'
namedataASCD_M = r'dataASCD_M.xlsx'
rutedataASCD_M = os.path.join(folder,namedataASCD_M)

In [12]:
dataASCD_M = pd.read_excel(rutedataASCD_M,header=0)

In [13]:
filterMT = dataASCD_M.loc[:,'Maintenance type'] != 'COND'
dataASCD_1TM = dataASCD_M.loc[filterMT,:].reset_index(drop=True)

In [14]:
dataASCD_1TM.loc[:,'Strategy'] = dataASCD_1TM.loc[:,'Frequency'].astype(int).astype(str) + dataASCD_1TM.loc[:,'Unidad Periodo']

dataASCD_1TM.loc[:,'Man hour'] = dataASCD_1TM.loc[:,'Time']*dataASCD_1TM.loc[:,'Amount Craft']

In [15]:
# Adding the pivot column to project
dataASCD_1TM.loc[:,'pivot'] = dataASCD_1TM.loc[:,'Asset tag'] + '-' + dataASCD_1TM.loc[:,'Strategy'] 
#dataASCD_1TP.loc[:,'pivot'] = dataASCD_1TP.loc[:,'Asset tag'] + '-' + dataASCD_1TP.loc[:,'Strategy']

In [16]:
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo','Strategy', 'Date task done']

In [17]:
projectionM = dataASCD_1TM.loc[:,['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']].groupby(['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']).size().reset_index()
projectionM.drop(columns = {0},inplace =True)
projectionM.loc[:,'Date plan'] = projectionM.loc[:,'Last date task done']
[row,column] = projectionM.shape

In [18]:
%%time
planDatesM = pd.DataFrame(columns = projectionM.columns)
projectionflag = projectionM.copy()
for m in range(row):
    while projectionflag.loc[m,'Date plan'] < date(2023,1,1):
        hour2days = round(projectionflag.loc[m,'Frequency']/projectionflag.loc[m,'Hour day'],0)
        if projectionflag.loc[m,'Unidad Periodo'] == 'H':
            delta = relativedelta(days = hour2days)
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'S':
            delta = relativedelta(weeks = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'M':
            delta = relativedelta(months = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'A':
            delta = relativedelta(years = projectionflag.loc[m,'Frequency'])
        projectionflag.loc[m,'Date plan'] = projectionflag.loc[m,'Date plan'] + delta
        if projectionflag.loc[m,'Date plan'] < date(2023,1,1):
            planDatesM = planDatesM.append(projectionflag.loc[m,:],ignore_index=True)
        else:
            pass
planDatesM.loc[:,'pivot'] = planDatesM.loc[:,'Asset tag'] + '-' + planDatesM.loc[:,'Strategy']

Wall time: 2min 24s


In [19]:
planDatesM.shape

(13025, 8)

In [20]:
%%time
dataASCD_1TMPlan = pd.merge(dataASCD_1TM,planDatesM.loc[:,['pivot','Date plan']],how='left',on='pivot')
dataASCD_1TMPlan.to_excel('dataASCD_1TMPlan.xlsx', encoding='utf-8-sig',index=False)

Wall time: 1min 41s


In [21]:
dataASCD_1TMPlan.shape

(59629, 30)

In [22]:
dataASCD_1TMPlan.columns

Index(['Asset tag', 'Name', 'Cod sap', 'WBS_1_EN', 'WBS_2_EN', 'WBS_3_EN',
       'WBS_4_EN', 'Responsable', 'Hour day', 'Criticidad', 'Criticidad Name',
       'key_system', 'System', 'key_component', 'Component', 'key_dutie',
       'Primary task', 'Primary task detail activity', 'Maintenance type',
       'Specialist', 'Constraint', 'Time', 'Amount Craft', 'Frequency',
       'Unidad Periodo', 'Last date task done', 'Man hour', 'Strategy',
       'pivot', 'Date plan'],
      dtype='object')